In [94]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import typing

In [83]:
def futures_header():
    """網頁瀏覽時, 所帶的 request header 參數, 模仿瀏覽器發送 request"""
    return {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36 Edg/116.0.1938.76",
        "content-type":"text/html; charset=UTF-8"
    }

In [116]:
def year_stats_columes(soup: BeautifulSoup) -> typing.List[str]:
    col_list = soup.find("body", class_="toolbar-enabled bg-light").find("section", class_="bg-dark text-light py-md-5 py-0").\
        find("div", class_='container-fluid').find("div", id="lock-table-scroll").find("table", id="main-table").\
        find("thead", class_="bg-deepgray").find("tr").select("th")
    columes = [col.get_text() for col in col_list]
    return columes

In [117]:
def year_stats_data(soup: BeautifulSoup) -> typing.List[typing.List[str]]:
    stats_list = soup.find("body", class_="toolbar-enabled bg-light").find("section", class_="bg-dark text-light py-md-5 py-0").\
        find("div", class_='container-fluid').find("div", id="lock-table-scroll").find("table", id="main-table").\
        find("tbody").find_all("tr")
    player_total_stats = []
    for stats in stats_list:
        name = stats.find("th").get_text()
        all_stats = stats.find_all("td", {'data-total': True})
        total_stats = [data['data-total'] for data in all_stats]

        total_stats.insert(0, name)
        player_total_stats.append(total_stats)
    return player_total_stats

In [145]:
def convert_datatype(data: pd.DataFrame) -> pd.DataFrame:
    data.drop(columns=['兩分%', '三分%', '罰球%', '籃板'], inplace=True)
    data.columns = ['name', 'number', 'team', 'games', 'two_points_field_goals_made', 'two_points_field_goals_attempt',\
        'three_points_field_goals_made', 'three_points_field_goals_attempt', 'free_throws_made', 'free_throws_attempt', 'points',\
        'offense_rebounds', 'defense_rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls', 'playing_seconds']
    data[['games', 'playing_seconds', 'two_points_field_goals_made', 'two_points_field_goals_attempt',\
        'three_points_field_goals_made', 'three_points_field_goals_attempt', 'free_throws_made', 'free_throws_attempt', 'points',\
        'offense_rebounds', 'defense_rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls']] \
    = data[['games', 'playing_seconds', 'two_points_field_goals_made', 'two_points_field_goals_attempt',\
        'three_points_field_goals_made', 'three_points_field_goals_attempt', 'free_throws_made', 'free_throws_attempt', 'points',\
        'offense_rebounds', 'defense_rebounds', 'assists', 'steals', 'blocks', 'turnovers', 'personal_fouls']].apply(pd.to_numeric)
    
    return data

In [151]:
def convert_mins_to_seconds(data: pd.DataFrame) -> pd.DataFrame:
    data['playing_seconds'] = data['時間 (分)'].apply(lambda x: int(x.split(":")[0])*60+int(x.split(":")[1]))
    data.drop(columns=['時間 (分)'], inplace=True)
    return data

In [306]:
r = requests.get("https://pleagueofficial.com/game/74")


In [323]:
soup = BeautifulSoup(r.text, "html.parser")
type(soup)

bs4.BeautifulSoup

In [324]:
data={'season':1, 'game':2, 'home_team_id':3, 'away_team_id':4}

In [326]:
pd.DataFrame(data=data, index=[0])

,season,game,home_team_id,away_team_id
0,1,2,3,4


In [322]:
soup.find("div", "col-lg-7 col-12 text-left align-self-center").find("h6", "pb-0 mb-0 mt-2 fs14").get_text().strip("\n ")

'臺北富邦勇士'

In [113]:
col_list = soup.find("body", class_="toolbar-enabled bg-light").find("section", class_="bg-dark text-light py-md-5 py-0").\
    find("div", class_='container-fluid').find("div", id="lock-table-scroll").find("table", id="main-table").\
    find("thead", class_="bg-deepgray").find("tr").select("th")
# stats_list = soup.find("body", class_="toolbar-enabled bg-light").find("section", class_="bg-dark text-light py-md-5 py-0").\
#     find("div", class_='container-fluid').find("div", id="lock-table-scroll").find("table", id="main-table").\
#     find("tbody").find_all("tr")

In [114]:
columes = [col.get_text() for col in col_list]

In [87]:
player_total_stats = []
for stats in stats_list:
    name = stats.find("th").get_text()
    all_stats = stats.find_all("td", {'data-total': True})
    total_stats = [data['data-total'] for data in all_stats]

    total_stats.insert(0, name)
    player_total_stats.append(total_stats)

In [91]:
total_data = pd.DataFrame(player_total_stats, columns=columes)

In [93]:
total_data.head()

,球員,背號,球隊,出賽次數,時間 (分),兩分命中,兩分出手,兩分%,三分命中,三分出手,...,罰球%,得分,攻板,防板,籃板,助攻,抄截,阻攻,失誤,犯規
0,鑀倫,19,高雄鋼鐵人,9,305:46,25,56,44.64,31,91,...,89.19,176,4,42,46,30,21,5,17,26
1,奧帝,11,新北國王,7,237:10,19,39,48.72,19,54,...,91.67,117,6,53,59,20,8,12,15,24
2,特壞,6,新竹街口攻城獅,18,738:31,96,196,48.98,35,120,...,66.04,367,30,96,126,66,28,6,41,28
3,楊敬敏,6,新北國王,39,1113:13,121,278,43.53,67,199,...,79.88,578,18,120,138,102,29,0,77,49
4,林俊吉,11,福爾摩沙台新夢想家,40,1132:42,93,229,40.61,85,217,...,80.71,554,29,126,155,174,31,1,82,75


In [56]:
l = stats_list[0].find_all("td", {'data-avg': True, 'data-total': True})
print([data['data-avg'] for data in l])

['19', '高雄鋼鐵人', '9', '33:58', '2.78', '6.22', '44.64', '3.44', '10.11', '34.07', '3.67', '4.11', '89.19', '19.56', '0.44', '4.67', '5.11', '3.33', '2.33', '0.56', '1.89', '2.89']


In [106]:
stats_list = stats_list[:10]
stats = [stats.get_text().split("\n")[1:-1] for stats in stats_list]
# len(stats.split("\n")[1:-1])
print(stats)

[['鑀倫', '19', '高雄17直播鋼鐵人', '9', '33:58', '2.78', '6.22', '44.64%', '3.44', '10.11', '34.07%', '3.67', '4.11', '89.19%', '19.56', '0.44', '4.67', '5.11', '3.33', '2.33', '0.56', '1.89', '2.89'], ['奧帝', '11', '新北國王', '7', '33:52', '2.71', '5.57', '48.72%', '2.71', '7.71', '35.19%', '3.14', '3.43', '91.67%', '16.71', '0.86', '7.57', '8.43', '2.86', '1.14', '1.71', '2.14', '3.43'], ['特壞', '6', '新竹攻城獅', '18', '41:1', '5.33', '10.89', '48.98%', '1.94', '6.67', '29.17%', '3.89', '5.89', '66.04%', '20.39', '1.67', '5.33', '7', '3.67', '1.56', '0.33', '2.28', '1.56'], ['楊敬敏', '6', '新北國王', '39', '28:32', '3.1', '7.13', '43.53%', '1.72', '5.1', '33.67%', '3.46', '4.33', '79.88%', '14.82', '0.46', '3.08', '3.54', '2.62', '0.74', '0', '1.97', '1.26'], ['林俊吉', '11', '福爾摩沙夢想家', '40', '28:19', '2.33', '5.73', '40.61%', '2.13', '5.43', '39.17%', '2.83', '3.5', '80.71%', '13.85', '0.73', '3.15', '3.88', '4.35', '0.78', '0.03', '2.05', '1.88'], ['李德威', '26', '福爾摩沙夢想家', '39', '21:20', '2.33', '5.18', '45.

In [69]:
# for col in col_list:
#     print(col.get_text())
cols = [col.get_text() for col in col_list]

NameError: name 'col_list' is not defined

In [84]:
from bs4 import BeautifulSoup

# HTML標籤字符串
html_tag = '<th class="w43" title="playername">球員</th>'

# 使用Beautiful Soup解析HTML
soup = BeautifulSoup(html_tag, 'html.parser')

# 提取標籤中的文本內容
text = soup.get_text()

# 打印提取的文本
print(text)


球員


In [17]:
print(soup)

<!DOCTYPE html>

<html lang="zh-TW">
<head>
<meta charset="utf-8"/>
<link href="/fav/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/fav/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
<link href="/fav/apple-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
<link href="/fav/apple-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
<link href="/fav/apple-icon-114x114.png" rel="apple-touch-icon" sizes="114x114"/>
<link href="/fav/apple-icon-120x120.png" rel="apple-touch-icon" sizes="120x120"/>
<link href="/fav/apple-icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
<link href="/fav/apple-icon-152x152.png" rel="apple-touch-icon" sizes="152x152"/>
<link href="/fav/apple-icon-180x180.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/fav/android-icon-192x192.png" rel="icon" sizes="192x192" type="image/png"/>
<link href="/fav/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/fav/favicon-96x96.png" rel="

In [22]:
player_data = soup.find('div', {'class': 'col-md-4 col-6 px-1 mb-2'})

In [23]:
player_data

<div class="col-md-4 col-6 px-1 mb-2">
<select class="bg-deepblue p-2 text-light col-12 fs14 border-0" id="season_name" onchange="selectChange();">
<option value="2020-21">2020-21</option><option value="2021-22">2021-22</option><option selected="" value="2022-23">2022-23</option> </select>
</div>

In [14]:
soup.select("div.col-sm-12 ")

[]